In [1]:
import board
import busio
import time
import adafruit_bno055

In [25]:
def get_GPS_data(device_address=0x42, device_register=0xFF, time_limit=60, verbose=False):
    """This function pulls data from a Ublox ZED-F9P and returns an array of NMEA messages
    
    First the function connects to the GPS via I2C at the specified address and interal
    register address. For the ZED-F9P the default device address is 0x42 and the default
    internal register is at 0xFF. After collecting the bytes from the device, the bytes
    are processed into readable char. Data collection starts once a $ is found and ends
    when the byte returned is equivalent to 255.
    
    This function takes between 0.3 seconds to 2 seconds to collect the data. The bottle
    neck is the data acquisition on the device and changing the code did not significantly
    impact run time.
    
    Args:
        device_address: a hex number representing the device address. The default value is
            0x42.
        device_register: a hex representing the interal data register address. The default
            value is 0xFF.
        time_limit: an int or float that puts a limit on how long this function will run
            before it quits collecting data. This is to prevent an infinite loop in case
            the device hangs up.
        verbose: a boolean specifying whether to print progress and time
            
    Returns:
        device_output: an array of strings representing NMEA messages
    """
    i2c_hook = busio.I2C(board.SCL, board.SDA)
    while not i2c_hook.try_lock():
        pass
    
    buffer = bytearray(1)
    data_initialized = False
    data_finished = False
    data_array = []
    start_time = time.perf_counter()
    run_time = 0
    while((not data_initialized or not data_finished) and run_time < time_limit):
        try:
            i2c_hook.writeto(device_address, bytes([device_register]))
            i2c_hook.readfrom_into(device_address, buffer)
            char_arr = [chr(x) for x in buffer]
            cur_char = char_arr[0]
            data_length = len(data_array)
            if(cur_char == '$' and not data_initialized):
                if verbose:
                    print("Data Collection Started")
                data_initialized = True
            if((data_initialized and int(buffer[0]) == 255) and data_length > 1): 
                if verbose:
                    print("Data Collection Finished")
                data_finished = True
            
            if(data_initialized and not data_finished):
                data_array.append(cur_char)
                
        except OSError:
            continue
        cur_time = time.perf_counter()
        run_time = float(cur_time - start_time)
        
    if verbose:
        print(run_time)
    data_output = "".join(data_array).split("$")
    data_output = data_output[1:]
    
    return data_output

In [28]:
raw_gps = get_GPS_data(verbose=True)
for message in raw_gps:
    print(message)

Data Collection Started
Data Collection Finished
1.0941297099998337
GNRMC,205742.00,A,3725.60333,N,12209.14620,W,0.064,,250620,,,A,V*0E

GNVTG,,T,,M,0.064,N,0.118,K,A*37

GNGGA,205742.00,3725.60333,N,12209.14620,W,1,12,0.97,-3.2,M,-30.0,M,,*56

GNGSA,A,3,25,29,05,12,,,,,,,,,2.06,0.97,1.81,1*09

GNGSA,A,3,82,79,80,81,66,78,,,,,,,2.06,0.97,1.81,2*0A

GNGSA,A,3,,,,,,,,,,,,,2.06,0.97,1.81,3*01

GNGSA,A,3,30,29,20,,,,,,,,,,2.06,0.97,1.81,4*0C

GPGSV,2,1,08,02,33,051,16,05,53,090,23,12,50,163,18,18,26,243,,1*66

GPGSV,2,2,08,25,72,221,24,26,00,323,,29,55,323,29,31,15,296,,1*66

GPGSV,2,1,08,02,33,051,,05,53,090,,12,50,163,,18,26,243,,*58

GPGSV,2,2,08,25,72,221,,26,00,323,,29,55,323,,31,15,296,,*5A

GLGSV,3,1,10,65,18,040,21,66,29,108,31,67,07,155,29,78,12,186,08,1*71

GLGSV,3,2,10,79,47,231,12,80,35,309,25,81,52,013,27,82,38,297,30,1*72

GLGSV,3,3,10,83,02,266,,88,16,062,,1*71

GLGSV,3,1,10,65,18,040,,66,29,108,,67,07,155,,78,12,186,,*42

GLGSV,3,2,10,79,47,231,,80,35,309,,81,52,013,,82,38,

In [37]:
def get_IMU_data(i2c_connection=None, sensor=None):
    """This function captures IMU data from a BNO055 9DOF sensor over i2c.
    
    The initialization of the IMU takes some time. However, once data
    acquisition begins, it's acquiring new data at around 55Hz.
    
    Args:
        i2c_connection: A busio.I2C class. This allows you to create
            a connection before running this function allowing faster
            data acquisition in a loop
        sensor: An adafruit_bno055.BNO055_I2C class. Allows connection
            to the IMU for faster execution of this function inside of
            a loop
    
    Returns:
        IMU_data: a list of IMU data (a mix of floats and tuples of floats).
        IMU_labels: a list of labels that correspond to the IMU_data.
        IMU_dict: a dictionary pairing labels and data together for easy access
    """
    if(i2c_connection):
        i2c = i2c_connection
    else:
        i2c = busio.I2C(board.SCL, board.SDA)
        
    if(not sensor):
        sensor = adafruit_bno055.BNO055_I2C(i2c)
    
    IMU_data = [sensor.temperature, sensor.acceleration,
                sensor.magnetic, sensor.gyro, sensor.euler,
                sensor.quaternion, sensor.linear_acceleration,
                sensor.gravity]
    IMU_labels = ["temperature", "acceleration", "magnetic",
                  "gyro", "euler", "quaternion", "linear acceleration", "gravity"]
    IMU_dict = dict(zip(IMU_labels, IMU_data))
    
    return IMU_data, IMU_labels, IMU_dict

In [41]:
i2c_connection = busio.I2C(board.SCL, board.SDA)
sensor = adafruit_bno055.BNO055_I2C(i2c_connection)
for i in range(15):
    start_time = time.perf_counter()
    _, _, imu_dat = get_IMU_data(i2c_connection, sensor)
    print(str(imu_dat) + "\ntime for new data acquisition: " + str(time.perf_counter() - start_time) + "\n")

{'temperature': 30, 'acceleration': (0.0, 0.0, 0.0), 'magnetic': (-8.875, 6.5625, 17.75), 'gyro': (0.002181661564992912, -0.001090830782496456, 0.003272492347489368), 'euler': (0.0, 0.0, 0.0), 'quaternion': (0.0, 0.0, 0.0, 0.0), 'linear acceleration': (0.0, 0.0, 0.0), 'gravity': (0.0, 0.0, 0.0)}
time for new data acquisition: 0.019377691998670343

{'temperature': 30, 'acceleration': (0.92, 2.84, -9.18), 'magnetic': (-8.875, 6.5625, 17.75), 'gyro': (0.0, -0.002181661564992912, 0.011999138607461015), 'euler': (0.0, 0.0, 0.0), 'quaternion': (0.0, 0.0, 0.0, 0.0), 'linear acceleration': (0.0, 0.0, 0.0), 'gravity': (0.0, 0.0, 0.0)}
time for new data acquisition: 0.021084776999487076

{'temperature': 30, 'acceleration': (0.91, 2.86, -9.18), 'magnetic': (-8.875, 6.5625, 17.75), 'gyro': (0.0, -0.001090830782496456, 0.008726646259971648), 'euler': (0.0, 0.0, 0.0), 'quaternion': (0.0, 0.0, 0.0, 0.0), 'linear acceleration': (0.0, 0.0, 0.0), 'gravity': (0.0, 0.0, 9.8)}
time for new data acquisition